# Fraud Detection

In [2]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('train_internet.csv')

In [11]:
test = pd.read_csv('data/test_public.csv')

In [5]:
test = pd.read_csv('data/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'

In [6]:
test = pd.read_csv('./data/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/test.csv'

In [7]:
sample = pd.read_csv('sampleSubmission.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sampleSubmission.csv'

In [8]:
top_cities = ["NewYork","LosAngeles","Chicago","Houston","Phoenix","Philadelphia","SanAntonio","SanDiego","Dallas","SanJose","Austin","Jacksonville","FortWorth","Columbus","Indianapolis","Charlotte","SanFrancisco","Seattle","Denver","Washington","Nashville","OklahomaCity","ElPaso","Boston","Portland","LasVegas","Detroit","Memphis","Louisville","Baltimore","Milwaukee","Albuquerque","Tucson","Fresno","Sacramento","KansasCity","Mesa","Atlanta","Omaha","ColoradoSprings","Raleigh","LongBeach","VirginiaBeach","Miami","Oakland","Minneapolis","Tulsa","Bakersfield","Wichita","Arlington"]

In [565]:
train['incident_city'].value_counts()

Springfield    117
Arlington      110
Columbus       108
Northbend       96
Hillsdale       96
Riverwood       90
Northbrook      83
Name: incident_city, dtype: int64

In [566]:
# for col in train['incident_state']:
#     if col in top_state:
#         train['big_state'] = 2
#     elif col in second_state:
#         train['big_state'] = 1
#     else:
#         train['big_state'] = 0

In [567]:
test.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital-gains', 'capital-loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', '_c39'],
      dtype='object')

In [568]:
data = pd.concat([train, test], axis=0)

In [569]:
data['auto_model'].value_counts().count()

39

In [570]:
car_price = pd.read_excel('res.xlsx')
car_price.columns = ['auto_model', 'model_price']
car_price['model_price'].nunique()

22

In [571]:
data = pd.merge(data, car_price,on='auto_model', how='outer')

In [572]:
# data.drop(['auto_model'], axis=1, inplace=True)

把日期转换成更有意义的特征，列入星期几

In [573]:
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'], errors='coerce')
data['incident_date'] = pd.to_datetime(data['incident_date'], errors='coerce')

In [574]:
data['policy_bind_date_weekday'] = data['policy_bind_date'].dt.weekday
data['incident_date_weekday'] = data['incident_date'].dt.weekday

In [575]:
base_date = data['policy_bind_date'].min()
data['p_diff'] = (data['policy_bind_date'] - base_date).dt.days
data['i_diff'] = (data['incident_date'] - base_date).dt.days
data.drop(['policy_bind_date', 'incident_date'], axis=1, inplace=True)

In [576]:
data['pi_diff'] = data['p_diff'] - data['i_diff']

In [577]:
v = pd.DataFrame({
    'top_state': ["NY", "SC", "WV"],
    'second_state': ["NC", "VA", "chi"], 
    'third_state' : ["PA", "OH", "ttt"]
})

data['big_state'] = data['incident_state'].apply(lambda x: '3' if x in v['top_state'].values else '2' if x in v['second_state'].values 
                                             else '1' if x in v['third_state'].values else '0')

In [578]:
data.drop('incident_state', axis=1, inplace=True)

In [579]:
v = pd.DataFrame({
    'top_city': ["Springfield", "Arlington", "Columbus"],
    'second_city': ["Northbend", "Hillsdale", "chi"], 
    'third_city' : ["Riverwood", "Northbrook", "ttt"]
})

data['big_city'] = data['incident_city'].apply(lambda x: '3' if x in v['top_city'].values else '2' if x in v['second_city'].values 
                                             else '1' if x in v['third_city'].values else '0')

In [580]:
data.drop('incident_city', axis=1, inplace=True)

In [581]:
for col in data.select_dtypes(include=object).columns:
    # nqunique是一个方法，记得加括号
	print(col, data[col].nunique())

policy_state 3
policy_csl 3
insured_sex 2
insured_education_level 7
insured_occupation 14
insured_hobbies 20
insured_relationship 6
incident_type 4
collision_type 4
incident_severity 4
authorities_contacted 5
incident_location 1000
property_damage 3
police_report_available 3
auto_make 14
auto_model 39
big_state 3
big_city 3


In [582]:
column_name = []
unique_value = []

for col in data.select_dtypes(include=object).columns:
	column_name.append(col)
	unique_value.append(data[col].nunique())

In [583]:
df = pd.DataFrame()
df['col_name'] =  column_name
df['value'] = unique_value
df = df.sort_values('value', ascending=False)
df

,col_name,value
11,incident_location,1000
15,auto_model,39
5,insured_hobbies,20
4,insured_occupation,14
14,auto_make,14
3,insured_education_level,7
6,insured_relationship,6
10,authorities_contacted,5
9,incident_severity,4
7,incident_type,4


In [584]:
data['authorities_contacted']

0      Ambulance
1         Police
2         Police
3          Other
4      Ambulance
         ...    
995       Police
996       Police
997         Fire
998       Police
999       Police
Name: authorities_contacted, Length: 1000, dtype: object

In [585]:
temp = pd.DataFrame()

In [586]:
cat_columns = data.select_dtypes(include='O').columns
float_d = data.copy()

In [587]:
cat_l = list(cat_columns)
for i in cat_l:
    float_d.drop(i,axis=1, inplace=True)

In [588]:
from sklearn.preprocessing import LabelEncoder

for col in cat_columns:
	le = LabelEncoder()
	temp[col] = le.fit_transform(data[col])

temp['index'] = range(1, len(temp) + 1)
temp.set_index('index')

,policy_state,policy_csl,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,incident_type,collision_type,incident_severity,authorities_contacted,incident_location,property_damage,police_report_available,auto_make,auto_model,big_state,big_city
index,,,,,,,,,,,,,,,,,,
1,1,2,0,5,10,15,1,2,3,2,0,680,0,0,9,26,1,0
2,0,1,1,5,7,3,2,3,0,1,4,462,2,1,9,26,2,0
3,1,0,1,6,6,0,2,0,2,0,4,629,2,0,9,26,2,2
4,2,1,1,3,2,12,4,2,3,2,3,431,2,1,9,26,2,0
5,2,1,0,0,10,10,1,2,1,1,0,32,0,0,9,26,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,2,1,0,2,6,18,1,1,0,3,4,294,2,2,2,0,2,2
997,2,0,0,4,10,16,2,3,0,1,4,334,1,2,2,0,2,0
998,0,1,1,5,1,9,0,0,3,2,1,158,1,1,2,0,2,1


In [589]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name == 'model_price' or feature_name == 'policy_number':
            continue
        else:
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [590]:
float_d = normalize(float_d)
float_d['index'] = range(1, len(float_d) + 1)
float_d.set_index('index')

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,...,vehicle_claim,auto_year,fraud_reported,_c39,model_price,policy_bind_date_weekday,incident_date_weekday,p_diff,i_diff,pi_diff
index,,,,,,,,,,,,,,,,,,,,,
1,0.390397,0.400000,125591,0.333333,0.606303,0.545455,0.109207,0.598010,1.000000,0.913043,...,0.579821,0.25,0.0,NaN,-0.770666,0.500000,0.666667,0.938644,0.254237,0.940601
2,0.162839,0.222222,694662,0.333333,0.496122,0.636364,0.144940,0.000000,1.000000,0.130435,...,0.065165,0.35,0.0,NaN,-0.770666,0.166667,0.500000,0.840017,0.474576,0.840731
3,0.321503,0.333333,598554,0.000000,0.224189,0.090909,0.938457,0.367164,1.000000,0.652174,...,0.678953,0.95,1.0,NaN,-0.770666,0.333333,0.833333,0.004032,0.152542,0.008268
4,0.396660,0.422222,601042,0.000000,0.503450,0.090909,0.067144,0.615920,0.549955,0.608696,...,0.396276,0.90,0.0,NaN,-0.770666,0.333333,0.333333,0.704337,0.457627,0.705396
5,0.323591,0.355556,327488,0.333333,0.301092,0.090909,0.154214,0.831841,1.000000,0.000000,...,0.404705,0.10,0.0,NaN,-0.770666,0.000000,1.000000,0.142655,0.288136,0.145779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.175365,0.222222,630998,0.333333,0.423624,0.090909,0.228133,0.000000,0.730873,0.260870,...,0.061517,0.50,NaN,NaN,-0.598918,0.333333,0.500000,0.527354,0.711864,0.527089
997,0.217119,0.244444,544225,0.000000,0.383780,0.090909,0.930498,0.000000,0.369037,0.347826,...,0.041389,0.55,NaN,NaN,-0.598918,0.166667,0.000000,0.818657,0.661017,0.818211
998,0.344468,0.377778,427199,1.000000,0.504590,0.090909,0.179358,0.000000,0.642664,0.869565,...,0.459051,0.65,NaN,NaN,-0.598918,0.666667,0.333333,0.825087,0.220339,0.827459


In [591]:
data = pd.merge(temp,float_d,on='index')

In [592]:
data.to_excel('/Users/19723/Desktop/yyyyyy.xlsx')

In [558]:
data.drop(['index', '_c39'], axis=1, inplace=True)

In [559]:
train = data[data['fraud_reported'].notnull()]
test = data[data['fraud_reported'].isnull()]

训练模型

In [450]:
# mac安装失败的话就用conda install lightgbm

In [451]:
# from sklearn.model_selection import RandomizedSearchCV
# import lightgbm as lgb

# rs_params = {

#         'colsample_bytree': (0.5, 0.6, 1),
#         'learning_rate': (0.005, 0.1, 0.2, 0.3),
#         'reg_lambda': (0.25, 0.3, 0.5, 3, 5),
#         'max_depth': (-1, 2, 3, 5, 10),
#         'min_child_samples': (1, 3, 5, 9, 10),
#         'num_leaves': (20, 2**5-1, 2**5-1, 300, 400),
#         'reg_alpha' : (0.1, 0.25, 0.3, 0.5, 3, 5)
    
# }

# # Initialize a RandomizedSearchCV object using 5-fold CV-
# # 折15次，每次用100样本
# rs_cv = RandomizedSearchCV(estimator=lgb.LGBMClassifier(), param_distributions=rs_params, cv = 7, n_iter=300,verbose=1)

# # Train on training data-
# rs_cv.fit(train.drop(['fraud_reported'], axis=1), train['fraud_reported'],verbose=1)
# print(rs_cv.best_params_)
# print(rs_cv.best_score_)

Fitting 7 folds for each of 300 candidates, totalling 2100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\19723\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


KeyboardInterrupt: 

In [452]:
# model_lgb = lgb.LGBMClassifier(num_leaves = 300, 
#                                reg_alpha=5, 
#                                reg_lambda=0.5, 
#                                objective='binary', 
#                                max_depth=3, 
#                                learning_rate=0.3, 
#                                min_child_samples=5, 
#                                random_state=7777,n_estimators=2000,subsample=1, colsample_bytree=1,)

In [453]:
# model_lgb.fit(train.drop(['fraud_reported'], axis=1), train['fraud_reported'])

LGBMClassifier(colsample_bytree=1, learning_rate=0.3, max_depth=3,
               min_child_samples=5, n_estimators=2000, num_leaves=300,
               objective='binary', random_state=7777, reg_alpha=5,
               reg_lambda=0.5, subsample=1)

In [454]:
# y_pred = model_lgb.predict_proba(test.drop(['fraud_reported'], axis=1))

In [455]:
# train['fraud_reported'].mean()

0.25857142857142856

In [456]:
# sum(y_pred) / 300

array([0.77069713, 0.22930287])

In [457]:
# result = pd.read_csv('sampleSubmission.csv')
# # 调整矩阵形状
# result['fraud_reported'] = y_pred[:, 1]

In [459]:
# result['fraud_reported'].to_excel('/Users/19723/Desktop/yyyya.xlsx')

In [460]:
# result

,policy_number,fraud_reported
0,698589,0.587929
1,287489,0.507880
2,211578,0.040526
3,807369,0.045366
4,830878,0.053479
...,...,...
295,679370,0.059729
296,272330,0.101335
297,315631,0.069951
298,445195,0.036823


In [458]:
# result.to_csv('baseline_20220302.csv', index=False)

In [235]:
# t = pd.read_csv('train.csv')
# t

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,187,37,125591,2013-08-08,IN,500/1000,1000,1412.06,5000000,450947,...,?,57700,5770,5770,46160,Nissan,Maxima,2000,0,NaN
1,243,44,967713,1997-12-25,IL,250/500,500,809.11,0,600208,...,YES,51400,5140,10280,35980,Honda,Civic,1996,0,NaN
2,24,33,649082,1996-01-19,IL,500/1000,1000,1922.84,0,431277,...,NO,46800,4680,9360,32760,Jeep,Wrangler,2002,0,NaN
3,215,42,519312,2008-10-28,OH,500/1000,500,1848.81,0,435489,...,YES,68520,11420,5710,51390,Suburu,Legacy,2003,1,NaN
4,85,30,190588,2001-12-09,OH,100/300,1000,796.35,0,614166,...,YES,58960,5360,10720,42880,Ford,F150,2004,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,199,37,982871,1997-07-27,IN,250/500,500,1262.08,0,474615,...,NO,60170,10940,10940,38290,Nissan,Pathfinder,2011,1,NaN
696,232,43,751905,2001-05-16,OH,250/500,500,1483.91,8000000,431531,...,?,70600,7060,14120,49420,Volkswagen,Passat,2013,1,NaN
697,215,37,761189,2002-12-28,IN,100/300,500,1632.93,0,614417,...,YES,57120,9520,4760,42840,Mercedes,C300,2002,0,NaN
698,270,44,701521,2003-07-05,IL,500/1000,2000,1030.95,0,435985,...,NO,35900,7180,3590,25130,Audi,A3,2007,1,NaN


In [229]:
# temp = pd.DataFrame(columns=["months_as_customer"])
# temp = t["months_as_customer"] 
# t = temp

In [233]:
# func1 = lambda x: 1 if x == True else 0

In [236]:
# t["months_as_customer"] = t["months_as_customer"]  > 24
# t["months_as_customer"]  = t["months_as_customer"] .apply(func1)
# t

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,1,37,125591,2013-08-08,IN,500/1000,1000,1412.06,5000000,450947,...,?,57700,5770,5770,46160,Nissan,Maxima,2000,0,NaN
1,1,44,967713,1997-12-25,IL,250/500,500,809.11,0,600208,...,YES,51400,5140,10280,35980,Honda,Civic,1996,0,NaN
2,0,33,649082,1996-01-19,IL,500/1000,1000,1922.84,0,431277,...,NO,46800,4680,9360,32760,Jeep,Wrangler,2002,0,NaN
3,1,42,519312,2008-10-28,OH,500/1000,500,1848.81,0,435489,...,YES,68520,11420,5710,51390,Suburu,Legacy,2003,1,NaN
4,1,30,190588,2001-12-09,OH,100/300,1000,796.35,0,614166,...,YES,58960,5360,10720,42880,Ford,F150,2004,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,37,982871,1997-07-27,IN,250/500,500,1262.08,0,474615,...,NO,60170,10940,10940,38290,Nissan,Pathfinder,2011,1,NaN
696,1,43,751905,2001-05-16,OH,250/500,500,1483.91,8000000,431531,...,?,70600,7060,14120,49420,Volkswagen,Passat,2013,1,NaN
697,1,37,761189,2002-12-28,IN,100/300,500,1632.93,0,614417,...,YES,57120,9520,4760,42840,Mercedes,C300,2002,0,NaN
698,1,44,701521,2003-07-05,IL,500/1000,2000,1030.95,0,435985,...,NO,35900,7180,3590,25130,Audi,A3,2007,1,NaN
